In [1]:
# Flickrライブラリのインストール
!pip install flickrapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#  必要ライブラリのインポート
import os
import time
from urllib.request import urlretrieve
from flickrapi import FlickrAPI
from tqdm import tqdm

In [3]:
#APIキー情報
FLICKR_KEY = "YOUR_FLICKR_KEY"
FLICKR_SECRET = "YOUR_FLICKR_SECRET"

In [4]:
# 乗り物の名前
VEHICLES = ["train", "car", "bicycle"]
# 各乗り物をループしてデータを取得
for vehicle in tqdm(VEHICLES):
    # 保存フォルダの指定
    save_dir = os.path.join("datasets", vehicle)
    os.makedirs(save_dir, exist_ok=True)
    # Flickr APIの初期化
    flickr = FlickrAPI(FLICKR_KEY, FLICKR_SECRET, format="parsed-json")
    # 乗り物の名前を指定して100件の画像情報を取得
    result = flickr.photos.search(
        text=vehicle,
        per_page=100,
        media="photos",
        sort="relevance",
        safe_search=1,
        extras="url_q, licence",
    )
    # 画像情報から実際の画像ファイルを取得
    photos = result["photos"]
    for photo in photos["photo"]:
        url_q = photo["url_q"]  # 画像のURL
        filepath = os.path.join(save_dir, photo["id"] + ".jpg")  # 画像のダウンロード先
        urlretrieve(url_q, filepath)  # 画像を指定したパスにダウンロードして保存
        time.sleep(1)  # クローリング先のサーバに負荷を与えないように1秒処理を止める

100%|██████████| 3/3 [05:41<00:00, 113.99s/it]


In [5]:
!rm -rf datasets

!wget -O datasets.zip https://github.com/AxrossDev/sharing/blob/main/datasets.zip?raw=true

--2023-01-23 07:02:12--  https://github.com/AxrossDev/sharing/blob/main/datasets.zip?raw=true
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/AxrossDev/sharing/raw/main/datasets.zip [following]
--2023-01-23 07:02:12--  https://github.com/AxrossDev/sharing/raw/main/datasets.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/AxrossDev/sharing/main/datasets.zip [following]
--2023-01-23 07:02:12--  https://raw.githubusercontent.com/AxrossDev/sharing/main/datasets.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5378155

In [6]:
!unzip datasets.zip

Archive:  datasets.zip
   creating: datasets/
   creating: datasets/car/
  inflating: datasets/car/17134719895.jpg  
  inflating: datasets/car/14294491930.jpg  
  inflating: datasets/car/14031162231.jpg  
  inflating: datasets/car/25369728.jpg  
  inflating: datasets/car/19258084569.jpg  
  inflating: datasets/car/14281103828.jpg  
  inflating: datasets/car/14820368061.jpg  
  inflating: datasets/car/14872174930.jpg  
  inflating: datasets/car/15704744728.jpg  
  inflating: datasets/car/38211504142.jpg  
  inflating: datasets/car/14907523078.jpg  
  inflating: datasets/car/16706383974.jpg  
  inflating: datasets/car/42285650904.jpg  
  inflating: datasets/car/34574127886.jpg  
  inflating: datasets/car/2789161277.jpg  
  inflating: datasets/car/47410285732.jpg  
  inflating: datasets/car/5804872160.jpg  
  inflating: datasets/car/455836722.jpg  
  inflating: datasets/car/20836107913.jpg  
  inflating: datasets/car/42691647791.jpg  
  inflating: datasets/car/14113937014.jpg  
  inflatin

In [7]:
# 必要ライブラリのインポート
import glob
import os

import numpy as np
from PIL import Image
from tqdm import tqdm

In [8]:
VEHICLES = ["train", "car", "bicycle"] # 乗り物の名前
DATASET_DIR = os.path.join("datasets") # データセットのフォルダパス

In [9]:
datasets = []
labels = []
for index, label in enumerate(tqdm(VEHICLES)):
    # 画像の読み込み
    photos_dir = os.path.join(DATASET_DIR, label)
    filepaths = glob.glob(os.path.join(photos_dir, "*.jpg"))

    # 画像を順次処理してデータセットを作成
    for i, filepath in enumerate(filepaths):
        # 各乗り物のデータ数を100に揃える
        if i >= 100:
            break

        image = Image.open(filepath)  # 画像の読み込み
        image = image.convert("RGB")  # RGBの３色に変換（画像を数値で扱うため）
        image = image.resize((50, 50))  # 画像のサイズを統一
        dataset = np.asarray(image)  # 画像を数値の配列に変換
        datasets.append(dataset)  # データセットを追加
        labels.append(index)  # ラベルを追加
print(len(datasets))
print(len(labels))

100%|██████████| 3/3 [00:00<00:00,  9.92it/s]

300
300


In [10]:
import numpy as np
from sklearn import model_selection

In [11]:
# TensorFlowがデータを処理しやすいようにnumpyのarrayに変換
datasets = np.array(datasets)
labels = np.array(labels)

# データセットとラベルの両方を学習データとテストデータに分割
dataset_train, dataset_test, label_train, label_test = model_selection.train_test_split(
    datasets, labels
)
print(dataset_train.shape)
print(dataset_test.shape)
print(label_train.shape)
print(label_test.shape)

(225, 50, 50, 3)
(75, 50, 50, 3)
(225,)
(75,)


In [12]:
import numpy as np

In [13]:
# 分割したデータをファイルに保存
data = (dataset_train, dataset_test, label_train, label_test)
np.save(os.path.join(DATASET_DIR, "vehicle.npy"), data)

/usr/local/lib/python3.8/dist-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [14]:
import os	
import numpy as np

In [15]:
VEHICLES = ["train", "car", "bicycle"]  # 乗り物の名前
DATASET_DIR = os.path.join("datasets")  # データセットのフォルダパス

# データセットの読み込み
dataset_train, dataset_test, label_train, label_test = np.load(
    os.path.join(DATASET_DIR, "vehicle.npy"), allow_pickle=True
)

# 画素を0~1の範囲に変換（正規化）
dataset_train = dataset_train.astype("float") / 255
dataset_test = dataset_test.astype("float") / 255

print(dataset_train.max())
print(dataset_train.min())
print(dataset_test.max())
print(dataset_test.min())

1.0
0.0
1.0
0.0


In [16]:
from keras.utils import np_utils

In [17]:
# ラベルをone-hotエンコーディング
label_train = np_utils.to_categorical(label_train, len(VEHICLES))
label_test = np_utils.to_categorical(label_test, len(VEHICLES))

print(label_train)
print(label_test)

[[1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0.

In [18]:
from keras.layers import Activation, Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop

In [19]:
# モデルの定義
# 今回はKerasを用いた画像分類のサンプルコードを利用
# https://github.com/aidiary/keras-examples/blob/master/cnn/cifar10/cifar10.py
model = Sequential()
model.add(Conv2D(32, (3, 3), padding="same", input_shape=dataset_train.shape[1:]))

model.add(Activation("relu"))
model.add(Conv2D(32, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation("relu"))
model.add(Dropout(0.25))
model.add(Dense(3))
model.add(Activation("softmax"))

In [20]:
opt = RMSprop(lr=0.0001, decay=1e-6)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 50, 50, 32)        896       
                                                                 
 activation (Activation)     (None, 50, 50, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 48, 48, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 24, 24, 32)        0         
                                                        

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [21]:
# モデルのトレーニング
model.fit(dataset_train, label_train, batch_size=32, epochs=100)

# モデルの保存
model.save(os.path.join(DATASET_DIR, "vehicle_cnn.h5"))

Epoch 1/100
8/8 [==============================] - 10s 32ms/step - loss: 1.0996 - accuracy: 0.3556
Epoch 2/100
8/8 [==============================] - 0s 9ms/step - loss: 1.1049 - accuracy: 0.3822
Epoch 3/100
8/8 [==============================] - 0s 9ms/step - loss: 1.0645 - accuracy: 0.4756
Epoch 4/100
8/8 [==============================] - 0s 10ms/step - loss: 1.0495 - accuracy: 0.4800
Epoch 5/100
8/8 [==============================] - 0s 9ms/step - loss: 1.0371 - accuracy: 0.4578
Epoch 6/100
8/8 [==============================] - 0s 9ms/step - loss: 1.0044 - accuracy: 0.5644
Epoch 7/100
8/8 [==============================] - 0s 9ms/step - loss: 0.9903 - accuracy: 0.5378
Epoch 8/100
8/8 [==============================] - 0s 9ms/step - loss: 0.9847 - accuracy: 0.5289
Epoch 9/100
8/8 [==============================] - 0s 10ms/step - loss: 0.9310 - accuracy: 0.5644
Epoch 10/100
8/8 [==============================] - 0s 9ms/step - loss: 0.9424 - accuracy: 0.5600
Epoch 11/100
8/8 [=======

In [22]:
import os

import keras
import numpy as np
from keras.utils import np_utils

In [23]:
VEHICLES = ["train", "car", "bicycle"]  # 乗り物の名前
DATASET_DIR = os.path.join("datasets")  # データセットのフォルダパス

# データセットの読み込み
dataset_train, dataset_test, label_train, label_test = np.load(
    os.path.join(DATASET_DIR, "vehicle.npy"), allow_pickle=True
)

# 画素を0~1の範囲に変換（正規化）
dataset_test = dataset_test.astype("float") / 255

# ラベルをone-hotエンコーディング
label_test = np_utils.to_categorical(label_test, len(VEHICLES))

# モデルの読み込み
model = keras.models.load_model(os.path.join(DATASET_DIR, "vehicle_cnn.h5"))
scores = model.evaluate(dataset_test, label_test, verbose=1)

print("Loss: " + str(scores[0]))
print("Accuracy: " + str(scores[1]))

3/3 [==============================] - 0s 52ms/step - loss: 1.1247 - accuracy: 0.6267
Loss: 1.124741554260254
Accuracy: 0.6266666650772095
